In [1]:
!pip install pandas

!pip install chromedriver-autoinstaller

!pip install BeautifulSoup4
!pip install lxml
!pip install selenium
!pip install selenium --upgrade
!pip install webdriver-manager

In [59]:
import os
import sys
from datetime import datetime
import time
import requests
import re

import pandas as pd

import chromedriver_autoinstaller
import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [72]:
# headless mode

option = webdriver.ChromeOptions()
option.add_argument("--headless")

service = Service(executable_path='chromedriver.exe')
browser = webdriver.Chrome(service=service, options=option)

html = browser.page_source
soup = BeautifulSoup(html, 'lxml')

url = r"https://www.catch.co.kr/"

browser.maximize_window()
print(browser.get_window_size())

browser.get(url)


{'width': 800, 'height': 600}


In [73]:
from accounts import ct

ID = ct.ID
PW = ct.PW

In [74]:
# 로그인

browser.find_element(By.CSS_SELECTOR, "a.login").click()

browser.find_element(By.CSS_SELECTOR, "input#id_login").send_keys(ID)
time.sleep(1)
browser.find_element(By.CSS_SELECTOR, "input#pw_login").send_keys(PW)

browser.find_element(By.CSS_SELECTOR, "input#pw_login").send_keys(Keys.ENTER)

In [75]:
# 기업 검색 (input)
query = browser.find_element(By.CSS_SELECTOR, "input")

query.send_keys(input("리뷰 확인하기: "))
query.send_keys(Keys.ENTER)

In [76]:
### 팝업 뜨면 끄기 (예외 처리)

try : 
    browser.find_element(By.CSS_SELECTOR, 'button.today > span').click()
    time.sleep(2)

except : 
    pass

In [77]:
# 검색한 기업(최상위 노출) 클릭 
browser.find_element(By.CSS_SELECTOR, "p.name > a").click()
time.sleep(2)

In [78]:
# '현직자 리뷰' 탭 클릭
browser.find_element(By.XPATH, r'//*[@id="Contents"]/div[1]/div[1]/div[2]/ul/li[4]/a').click()
time.sleep(1)

In [79]:
# 핵심 리뷰만 조회 - 정규직/현직자가 3년 이내 작성한 리뷰
browser.find_element(By.CSS_SELECTOR, 'div.pd_type2 > div > a').click()
time.sleep(1)

In [80]:
# 긍정 리뷰만 조회
good_rv = browser.find_element(By.CSS_SELECTOR, "li:nth-child(1) > p:nth-child(4)").text
good_rv = good_rv.replace(good_rv[:5], "").replace('\n', ' ') 
good_rv

'1. 유연근무제 , 해피프라이데이 등 근무 자유로움 2. 상대적으로 높은 연봉 3. 기숙사 및 식대 제공으로 비용 절감 가능'

In [81]:
# 부정 리뷰만 조회
bad_rv = browser.find_element(By.CSS_SELECTOR, "li:nth-child(1) > p:nth-child(5)").text
bad_rv = bad_rv.replace(bad_rv[:4], "").replace('\n', ' ') 
bad_rv

'1. 반도체 업계 특성상 사이클을 탐 2. 팀바팀이지만 야근하는 경우도 많음 3. 이천이라는 위치적인 단점'

In [82]:
good_df = pd.DataFrame({"긍정적 평가" : [good_rv]})
good_df


,긍정적 평가
0,"1. 유연근무제 , 해피프라이데이 등 근무 자유로움 2. 상대적으로 높은 연봉 3...."


In [83]:
bad_df = pd.DataFrame({"부정적 평가" : [bad_rv]})
bad_df

,부정적 평가
0,1. 반도체 업계 특성상 사이클을 탐 2. 팀바팀이지만 야근하는 경우도 많음 3. ...


In [84]:
good_df.join(bad_df)

,긍정적 평가,부정적 평가
0,"1. 유연근무제 , 해피프라이데이 등 근무 자유로움 2. 상대적으로 높은 연봉 3....",1. 반도체 업계 특성상 사이클을 탐 2. 팀바팀이지만 야근하는 경우도 많음 3. ...


In [85]:
# Web Crawling

n_btn = browser.find_element(By.CSS_SELECTOR, 'a.ico.next')


good_lst = None
bad_lst = None


good_lst = []
bad_lst = []


i = 0 

pre_page = 0

while True : 

    time.sleep(1)

    # 현재 페이지의 숫자
    crt_btn = browser.find_element(By.CSS_SELECTOR, 'div.pd_type2 > p > a.selected')
    crt_page = int(crt_btn.text)

    # 다음 버튼 눌러도 같은 페이지의 경우 (마지막 페이지)
    if crt_page == pre_page:
        break


    for p in range(1, 5) : 

        grv = browser.find_element(By.CSS_SELECTOR, f"li:nth-child({p}) > p:nth-child(4)").text
        brv = browser.find_element(By.CSS_SELECTOR, f"li:nth-child({p}) > p:nth-child(5)").text

        grv_clean = grv.replace(grv[:5], "").replace('\n', ' ') 
        brv_clean = brv.replace(brv[:4], "").replace('\n', ' ') 

        good_lst.append(grv_clean)
        bad_lst.append(brv_clean)


    time.sleep(2)

    # WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, f"li:nth-child({i}) > p:nth-child(4)")))


# 다음 페이지로 이동하기 위한 버튼을 찾기.
    try:
        # 이전페이지의 번호를 갱신
        pre_page = crt_page

        # 다음 버튼 클릭
        n_btn.click()

        # 버튼 누르고나서 정보를 가져올때까지 슬립(몇초동안)걸어두기.
        time.sleep(1)

    except:
        break

browser.close()

good_lst = list(set(good_lst))
bad_lst = list(set(bad_lst))


In [86]:
len(good_lst), len(bad_lst)

(44, 44)

In [87]:
good_lst

['연봉 복지 교육시설이 좋다. 출퇴근 버스 잘되어있다. 나름 수평적인 문화에 같이 일하는 사람들도 합리적이라 워라벨 좋은거같다',
 '서울에서 출퇴근 가능한 대기업임. 복지도 나름 높아지고 있음',
 '눈치볼 필요없이 칼퇴가능. 복지도 다양하고 좋습니다. 편의식도 좋아요',
 '직무에 따라 다르지만 제조쪽만 아니면 다들 자유로운 분위기',
 '복지가 많이 개선되었다. 사내분위기도 달라져서 출퇴근이 유연근무이다',
 '복지가 좋으며 육아 휴직 등 자유로워 워킹맘들이 많으며 육아하기 좋아요',
 '체육 시설이 다양하다(헬스장, 농구장, 풋살장, 수영장, 골프장, 탁구, 배드민턴) 하루 4끼 제공해준다',
 '고향이 충북이라 집갈 때 편하다 수도권 사람들은 별로 좋아하지 않는다.',
 '다른 회사에 비해 돈은 상당히 많이 준다. 직원복지도 다른기업 대비 괜찮다',
 '복지 좋고 애 엄마가 되어도 맘 편히 다닐 수 있다. 휴가를 자유롭게 쓸 수 있다',
 '출근 퇴근 좋고 유연 근무자의 경우 좋으나 눈치 좀 보임',
 '기숙사, 3식무료, 통근버스 등등 복지가 좋고 확실히 대기업 다닌다는 네임벨류가 있다',
 '셔틀버스가 잘되어있습니다. 3끼 모두 제공하고 식사를 안먹으면 편의식을 탈 수 있습니다.',
 '기숙사, 통근버스, 식사, 의료비 등 기본 복지가 잘 되어있다',
 '복지혜택이 많다. 월급이 많다. 성과급이 많다',
 '점점 근무시간 및 근무환경이 좋아지고 있다. 성과급이 크다',
 '회사가 지속적으로 성장했고 성장 가능성이 있다. 교육 등의 많은 기회가 주어지고 개인이 마음만 먹으면 해당 분야의 최고 전문가 가능함. 금전적인 보상이 수원의 회사와 비슷함. 해당 분야가 전망이 좋으므로 커리어를 쌓아서 다른 분야 진출 용이함. 회사에 실력이 뛰어난 구성원들이 많이 있어서 배울 것이 많음.',
 '유연근무제 시행되어서 좋고 복지 좋아요. 휴가도 맘대로 쓸 수 있는 부서를 만나야 자유롭게 쓸 듯.',
 '성과급이 있고 복지가 좋다... 회사에서 집이 가깝다',
 

In [88]:
good_lst[-5:], bad_lst[-5:]

(['기숙사 기간없음, 식사 3식 무료, 복지포인트 사용, 사내 병원, 대출, 자녀 학자금, 사내 대학 등 다양한 복지',
  '복지가 좋고 여러가지 보상제도가 잘 되어있습니다',
  '매년 복지 포인트가 나오고 의복도 주기적으로 제공해주고 식사 잘나오고 자녀있는 사람들은 학자금도 지원해줘서 좋다. 부바부지만 수평적인 기업문화 같고 지시하는 사람도 거의 없는편',
  '눈치 x , 칼퇴 가능 업계 성장가능성이 높고 동종업계 대비 높은 연봉과 높은 연봉 인상률',
  '급여가 나쁘지않다 여러 복지가 많다. 복지포인트. 휴가쓰기 자유롭다'],
 ['삼성대비 인센티브 많이 안나옴 잘될때 안될때 차이가 크고 이천 너무 시골임',
  '체계가 없어 가이드가 자주 바뀐다. 개인 커리어 성장에 한계가 느껴진다. 잔업이 그다지 많지 않아서 수당을 얻기는 힘들다',
  '공장증설로 본사는 맨날 공사판이다. 회사 내 버스 트럭이 위험하다.',
  '꼰대 기질이 있는 상사가 많고 보수적 성향으로 답답한 조직 문화가 있다',
  '삼성전자에 비해 연봉 인상률이 낮으며, 교대근무자들의 위한 셔틀은 다양하지 않음.'])

In [89]:
good_df = pd.DataFrame({"긍정적 평가" : good_lst})
bad_df = pd.DataFrame({"부정적 평가" : bad_lst})

good_df.shape, bad_df.shape

((44, 1), (44, 1))

In [90]:
good_df.tail()

,긍정적 평가
39,"기숙사 기간없음, 식사 3식 무료, 복지포인트 사용, 사내 병원, 대출, 자녀 학자..."
40,복지가 좋고 여러가지 보상제도가 잘 되어있습니다
41,매년 복지 포인트가 나오고 의복도 주기적으로 제공해주고 식사 잘나오고 자녀있는 사람...
42,"눈치 x , 칼퇴 가능 업계 성장가능성이 높고 동종업계 대비 높은 연봉과 높은 연봉..."
43,급여가 나쁘지않다 여러 복지가 많다. 복지포인트. 휴가쓰기 자유롭다


In [91]:
bad_df.tail()

,부정적 평가
39,삼성대비 인센티브 많이 안나옴 잘될때 안될때 차이가 크고 이천 너무 시골임
40,체계가 없어 가이드가 자주 바뀐다. 개인 커리어 성장에 한계가 느껴진다. 잔업이 그...
41,공장증설로 본사는 맨날 공사판이다. 회사 내 버스 트럭이 위험하다.
42,꼰대 기질이 있는 상사가 많고 보수적 성향으로 답답한 조직 문화가 있다
43,"삼성전자에 비해 연봉 인상률이 낮으며, 교대근무자들의 위한 셔틀은 다양하지 않음."


In [92]:
pros_cons = good_df.join(bad_df)
pros_cons

,긍정적 평가,부정적 평가
0,연봉 복지 교육시설이 좋다. 출퇴근 버스 잘되어있다. 나름 수평적인 문화에 같이 일...,경쟁사를 이길 수 있을까? 경쟁사 대비 성과급을 많이 받거나 비슷하게라고 받으려면...
1,서울에서 출퇴근 가능한 대기업임. 복지도 나름 높아지고 있음,"부서마다, 직무마다 업무량이 다르고 회사 주변에 놀 만한 곳이 없다"
2,눈치볼 필요없이 칼퇴가능. 복지도 다양하고 좋습니다. 편의식도 좋아요,교대 근무를 함에도 불구하고 다른 기업보다 임금이 짠 편이다
3,직무에 따라 다르지만 제조쪽만 아니면 다들 자유로운 분위기,여자친구가 멀리 있는데 가끔 청주에 놀러올 때마다 힘들다고 한다.
4,복지가 많이 개선되었다. 사내분위기도 달라져서 출퇴근이 유연근무이다,월급을 안 올려주려한다. 임단협 좀 끝냅시다. 물가 오른만큼 월급도 올라야죠
5,복지가 좋으며 육아 휴직 등 자유로워 워킹맘들이 많으며 육아하기 좋아요,회사 지역이 안좋아서 접근성이 불편하다. 출퇴근 버스가 1시간 단위마다 있다. 통신...
6,"체육 시설이 다양하다(헬스장, 농구장, 풋살장, 수영장, 골프장, 탁구, 배드민턴)...",보수적이고 경직된 문화로 계급과 경력을 중시해요 고지식한 면이 좀 있습니다
7,고향이 충북이라 집갈 때 편하다 수도권 사람들은 별로 좋아하지 않는다.,부서에 따라 고생하는 사람 편한 사람 따로 있다 상사 잘못 만나면 큰일...
8,다른 회사에 비해 돈은 상당히 많이 준다. 직원복지도 다른기업 대비 괜찮다,"부서마다 차이가 심해 노는 부서는 끝없이 놀고 제조, 제조기술 같은 부서는 365일 바쁨"
9,복지 좋고 애 엄마가 되어도 맘 편히 다닐 수 있다. 휴가를 자유롭게 쓸 수 있다,지리적으로 멀다. 일부 직무는 매우 비선호 된다


In [93]:
# 2023-07-04 21:00 commit
# 2023-07-05 13:05 modified

## to .py file

In [37]:
# %%writefile catch.py

# pip install

# !pip install pandas

# !pip install chromedriver-autoinstaller

# !pip install BeautifulSoup4
# !pip install lxml
# !pip install selenium
# !pip install selenium --upgrade
# !pip install webdriver-manager



# import

import os
import sys
from datetime import datetime
import time
import requests
import re

import pandas as pd

import chromedriver_autoinstaller
import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



# ChromeBrowser - headless mode

option = webdriver.ChromeOptions()
option.add_argument#("--headless")

service = Service(executable_path='chromedriver.exe')
browser = webdriver.Chrome(service=service, options=option)

html = browser.page_source
soup = BeautifulSoup(html, 'lxml')

url = r"https://www.catch.co.kr/"

browser.maximize_window()
print(browser.get_window_size())

browser.get(url)



# 로그인

from accounts import ct

ID = ct.ID
PW = ct.PW


browser.find_element(By.CSS_SELECTOR, "a.login").click()

browser.find_element(By.CSS_SELECTOR, "input#id_login").send_keys(ID)
time.sleep(1)
browser.find_element(By.CSS_SELECTOR, "input#pw_login").send_keys(PW)

browser.find_element(By.CSS_SELECTOR, "input#pw_login").send_keys(Keys.ENTER)



# 기업 검색 (input)
query = browser.find_element(By.CSS_SELECTOR, "input")

query.send_keys(input())
query.send_keys(Keys.ENTER)

try : 
    browser.find_element(By.CSS_SELECTOR, 'button.today > span').click()
    time.sleep(2)

except : 
    pass


browser.find_element(By.CSS_SELECTOR, "p.name > a").click()
time.sleep(2)

browser.find_element(By.XPATH, r'//*[@id="Contents"]/div[1]/div[1]/div[2]/ul/li[4]/a').click()
time.sleep(1)

browser.find_element(By.CSS_SELECTOR, 'div.pd_type2 > div > a').click()
time.sleep(1)



# reviews Crawling


n_btn = browser.find_element(By.CSS_SELECTOR, 'a.ico.next')


good_lst = []
bad_lst = []


i = 0 

pre_page = 0

while True : 

    # 현재 페이지의 숫자
    crt_btn = browser.find_element(By.CSS_SELECTOR, 'div.pd_type2 > p > a.selected')
    crt_page = int(crt_btn.text)

    # 다음 버튼 눌러도 같은 페이지의 경우 (마지막 페이지)
    if crt_page == pre_page:
        break


    for p in range(1, 5) : 

        grv = browser.find_element(By.CSS_SELECTOR, f"li:nth-child({p}) > p:nth-child(4)").text
        brv = browser.find_element(By.CSS_SELECTOR, f"li:nth-child({p}) > p:nth-child(5)").text

        grv_clean = grv.replace(grv[:5], "").replace('\n', ' ') 
        brv_clean = brv.replace(brv[:4], "").replace('\n', ' ') 

        good_lst.append(grv_clean)
        bad_lst.append(brv_clean)


    time.sleep(2)

    # WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, f"li:nth-child({i}) > p:nth-child(4)")))


# 다음 페이지로 이동하기 위한 버튼을 찾기.
    try:
        # 이전페이지의 번호를 갱신
        pre_page = crt_page

        # 다음 버튼 클릭
        n_btn.click()

        # 버튼 누르고나서 정보를 가져올때까지 슬립(몇초동안)걸어두기.
        time.sleep(1)

    except:
        break


good_lst = list(set(good_lst))
bad_lst = list(set(bad_lst))



# List 2 DataFrame

good_df = pd.DataFrame({"긍정적 평가" : good_lst})
bad_df = pd.DataFrame({"부정적 평가" : bad_lst})

pros_cons = good_df.join(bad_df)        # 긍정 부정 병합


{'width': 1552, 'height': 832}


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=114.0.5735.199); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0039A813+48355]
	(No symbol) [0x0032C4B1]
	(No symbol) [0x00235358]
	(No symbol) [0x002387A1]
	(No symbol) [0x002399E1]
	(No symbol) [0x00239A80]
	(No symbol) [0x0026127B]
	(No symbol) [0x00261301]
	(No symbol) [0x0025BA3F]
	(No symbol) [0x0027A73C]
	(No symbol) [0x00259A36]
	(No symbol) [0x0027AA94]
	(No symbol) [0x0028C922]
	(No symbol) [0x0027A536]
	(No symbol) [0x002582DC]
	(No symbol) [0x002593DD]
	GetHandleVerifier [0x005FAABD+2539405]
	GetHandleVerifier [0x0063A78F+2800735]
	GetHandleVerifier [0x0063456C+2775612]
	GetHandleVerifier [0x004251E0+616112]
	(No symbol) [0x00335F8C]
	(No symbol) [0x00332328]
	(No symbol) [0x0033240B]
	(No symbol) [0x00324FF7]
	B